In [1]:
@file:DependsOn("com.google.adk:google-adk:0.3.0")
@file:DependsOn("com.google.adk:google-adk-dev:0.2.0")
@file:DependsOn("com.google.genai:google-genai:1.23.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-rx3:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.8.1")

In [2]:
import com.google.adk.agents.LlmAgent
import com.google.adk.agents.SequentialAgent
import com.google.adk.agents.ParallelAgent
import com.google.adk.agents.LoopAgent

import com.google.adk.models.Gemini
import com.google.genai.Client

import com.google.adk.runner.InMemoryRunner
import com.google.adk.sessions.Session
import com.google.adk.sessions.InMemorySessionService
import com.google.adk.memory.InMemoryMemoryService
import com.google.adk.events.Event
import com.google.adk.tools.AgentTool
import com.google.adk.tools.FunctionTool
import com.google.adk.tools.GoogleSearchTool
import com.google.adk.codeexecutors.BuiltInCodeExecutor

import com.google.genai.types.Content
import com.google.genai.types.Part

import java.util.UUID
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.toList
import kotlinx.coroutines.reactive.asFlow

import io.reactivex.rxjava3.core.Flowable;

In [3]:
val apiKey = System.getenv("GOOGLE_API_KEY")
if (apiKey == null) {
    throw IllegalStateException("Please set the GOOGLE_API_KEY environment variable.")
}

In [4]:
val geminiClient = Client.builder().apiKey(apiKey).build()

In [5]:
val geminiModel = Gemini(
    "gemini-2.5-flash-lite", // Your model name
    geminiClient
)

## 🤖 Section 2: What are Custom Tools?

**Custom Tools** are tools you build yourself using your own code and business logic. Unlike built-in tools that come ready-made with ADK, custom tools give you complete control over functionality.

**When to use Custom Tools?**

Built-in tools like Google Search are powerful, but **every business has unique requirements** that generic tools can't handle. Custom tools let you implement your specific business logic, connect to your systems, and solve domain-specific problems. ADK provides multiple custom tool types to handle these scenarios.

### 2.1: Building Custom Function Tools

#### Example: Currency Converter Agent

This agent can convert currency from one denomination to another and calculates the fees to do the conversion. The agent has two custom tools and follows the workflow:

1. **Fee Lookup Tool** - Finds transaction fees for the conversion (mock)
2. **Exchange Rate Tool** - Gets currency conversion rates (mock)
3. **Calculation Step** - Calculates the total conversion cost including the fees

<img src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day2/currency-agent.png" width="600" alt="Currency Converter Agent">

### 🤔 2.2: How to define a Tool?

**Any Python function can become an agent tool** by following these simple guidelines:

1. Create a Python function
        2. Follow the best practices listed below
        3. Add your function to the agent's `tools=[]` list and ADK handles the rest automatically.


#### 🏆 ADK Best Practices in Action

Notice how our tools follow ADK best practices:

**1. Dictionary Returns**: Tools return `{"status": "success", "data": ...}` or `{"status": "error", "error_message": ...}`
**2. Clear Docstrings**: LLMs use docstrings to understand when and how to use tools
**3. Type Hints**: Enable ADK to generate proper schemas (`str`, `dict`, etc.)
**4. Error Handling**: Structured error responses help LLMs handle failures gracefully

These patterns make your tools reliable and easy for LLMs to use correctly.

👉 Let's see this in action with our first tool:

In [6]:
object PaymentMethodTools {
    fun getFeeForPaymentMethod(method: String): Map<String, Any> {
        // This simulates looking up a company's internal fee structure via some API
        val feeDatabase = mapOf(
            "platinum credit card" to 0.02,  // 2%
            "gold debit card" to 0.035, // 3.5%
            "bank transfer" to 0.01    // 1%
        )

        val fee = feeDatabase[method.lowercase()]

        return if (fee != null) {
            mapOf("status" to "success", "fee_percentage" to fee)
        } else {
            mapOf(
                "status" to "error",
                "error_message" to "Payment method '$method' not found"
            )
        }
    }
}

println("✅ PaymentMethodTools object with getFeeForPaymentMethod() created")
println("💳 Test: ${PaymentMethodTools.getFeeForPaymentMethod("platinum credit card")}")

✅ PaymentMethodTools object with getFeeForPaymentMethod() created
💳 Test: {status=success, fee_percentage=0.02}


In [7]:
object ExchangeRateTools {
    fun getExchangeRate(baseCurrency: String, targetCurrency: String): Map<String, Any> {
        // Static data simulating a live exchange rate API
        val rateDatabase = mapOf(
            "usd" to mapOf(
                "eur" to 0.93,   // Euro
                "jpy" to 157.50, // Japanese Yen
                "inr" to 83.58   // Indian Rupee
            )
        )

        val base = baseCurrency.lowercase()
        val target = targetCurrency.lowercase()

        val rate = rateDatabase[base]?.get(target)

        return if (rate != null) {
            mapOf("status" to "success", "rate" to rate)
        } else {
            mapOf(
                "status" to "error",
                "error_message" to "Unsupported currency pair: $baseCurrency/$targetCurrency"
            )
        }
    }
}
println("✅ ExchangeRateTools object with getExchangeRate() created")
println("💳 Test: ${ExchangeRateTools.getExchangeRate("USD", "eur")}")

✅ ExchangeRateTools object with getExchangeRate() created
💳 Test: {status=success, rate=0.93}


Now let's create our currency agent. Pay attention to how the agent's instructions reference the tools:

**Key Points:**
- The `tools=[]` list tells the agent which functions it can use
- Instructions reference tools by their exact function names (e.g.,
`get_fee_for_payment_method()`)
- The agent uses these names to decide when and how to call each tool

In [8]:
val exchangeRateTool = FunctionTool.create(
    ExchangeRateTools,
    "getExchangeRate"
)

val paymentMethodTool = FunctionTool.create(
    PaymentMethodTools,
    "getFeeForPaymentMethod"
)

val currencyAgent = LlmAgent.builder()
    .name("currency_agent")
    .model(geminiModel) // Re-using the model from the first cell
    .instruction("""You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `getFeeForPaymentMethod()` to find transaction fees
    2. Use `getExchangeRate()` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `getFeeForPaymentMethod` and `getExchangeRate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.""")
    .outputKey("current_story")
    .tools(listOf(exchangeRateTool, paymentMethodTool))
    .build()

println("✅ Currency agent created with custom function tools")
println("🔧 Available tools:")
println("  • getFeeForPaymentMethod - Looks up company fee structure")
println("  • getExchangeRate - Gets current exchange rates")

✅ Currency agent created with custom function tools
🔧 Available tools:
  • getFeeForPaymentMethod - Looks up company fee structure
  • getExchangeRate - Gets current exchange rates


In [9]:
var runner: InMemoryRunner = InMemoryRunner(currencyAgent, "Currency Flow")

In [10]:
val prompt = "I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?"
var userMsg: Content? = Content.fromParts(Part.fromText(prompt))

In [11]:
var session: Session? = runner
    .sessionService()
    .createSession("Currency Flow", "trader")
    .blockingGet()

In [12]:

println("Agent response ->")
var resultText = ""
runBlocking {
    var events = runner.runAsync("trader", session.id(), userMsg)
        .asFlow()
        .toList()

    events.forEach {
        // println(event.stringifyContent())
            event -> event
        .content().get()
        .parts().ifPresent { parts:List<Part> -> parts
            .forEach {
                if (it.text().isPresent) {
                    resultText += it.text().get()
                }
            }
        }
    }
}

MIME(
    "text/markdown" to resultText
)

Agent response ->


The fee for using a Platinum Credit Card is 2%. The exchange rate from USD to EUR is 0.93.

Here's the breakdown:

1. Fee amount: 500 USD * 0.02 = 10 USD
2. Amount after fee: 500 USD - 10 USD = 490 USD
3. Final amount in EUR: 490 USD * 0.93 = 455.7 EUR

You will receive 455.7 Euros.

**Excellent!** Our agent now uses custom business logic with structured responses.

## 💻 Section 3: Improving Agent Reliability with Code

The agent's instruction says *"calculate the final amount after fees"* but LLMs aren't always reliable at math. They might make calculation errors or use inconsistent formulas.

##### 💡 **Solution:** Let's ask our agent to generate a Python code to do the math, and run it to give us the final result! Code execution is much more reliable than having the LLM try to do math in its head!

<img src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day2/enhanced-currency-agent.png" width="800" alt="Enhanced Currency Converter Agent">

### 3.1 Built-in Code Executor

ADK has a built-in Code Executor capable of running code in a sandbox. **Note:** This uses Gemini's Code Execution capability.

Let's create a `calculation_agent` which takes in a Python code and uses the `BuiltInCodeExecutor` to run it.

In [13]:
import com.google.adk.codeexecutors.ContainerCodeExecutor

val calculationAgent = LlmAgent.builder()
    .name("calculation_agent")
    .model(geminiModel)
    .description("An agent that generates code to perform a certain calculation")
    .instruction("""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.

     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.

     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.

    Failure to follow these rules will result in an error.""")
    .codeExecutor(BuiltInCodeExecutor())
    .build()

println("✅ Code executor agent created")

✅ Code executor agent created


### 3.2: Update the Agent's instruction and toolset

We'll do two key actions:

1. **Update the `currency_agent`'s instructions to generate Python code**
- Original: "*Calculate the final amount after fees*" (vague math instructions)
- Enhanced: "*Generate a Python code to calculate the final amount .. and use the `calculation_agent` to run the code and compute final amount*"

2. **Add the `calculation_agent` to the toolset**

        ADK lets you use any agent as a tool using `AgentTool`.

- Add `AgentTool(agent=calculation_agent)` to the tools list
        - The specialist agent appears as a callable tool to the root agent

        Let's see this in action:

In [15]:
val enhancedCurrencyAgent = LlmAgent.builder()
    .name("enhanced_currency_agent")
    .model(geminiModel)
    // Updated instruction
    .description("test")
    .instruction("""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.
          For any currency conversion request:

           1. Get Transaction Fee: Use the getFeeForPaymentMethod() tool to determine the transaction fee.
           2. Get Exchange Rate: Use the getExchangeRate() tool to get the currency conversion rate.
           3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
           4. Calculate Final Amount (CRITICAL): You are ABSOLUTELY PROHIBITED from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This code will use the fee information from step 1 and the exchange rate from step 2.
           5. Provide Detailed Breakdown: In your summary, you must:
               * State the final converted amount returned by the calculation_agent.
               * Explain how the result was calculated, including:
                   * The fee percentage and the fee amount in the original currency.
                   * The amount remaining after deducting the fee.
                   * The exchange rate applied.
    """)
    .tools(
        paymentMethodTool,
        exchangeRateTool,
        AgentTool.create(calculationAgent)
    )
    .build()

println("✅ Enhanced currency agent created")
println("🎯 New capability: Delegates calculations to specialist agent")
println("🔧 Tool types used:")
println("  • Function Tools (fees, rates)")
println("  • Agent Tool (calculation specialist)")

✅ Enhanced currency agent created
🎯 New capability: Delegates calculations to specialist agent
🔧 Tool types used:
  • Function Tools (fees, rates)
  • Agent Tool (calculation specialist)


In [16]:
var enhancedRunner: InMemoryRunner = InMemoryRunner(enhancedCurrencyAgent, "Enhanced Currency Flow")

In [17]:
val newPrompt = "Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation"

In [18]:
var newUserMsg: Content? = Content.fromParts(Part.fromText(newPrompt))

In [19]:
var newSession: Session? = enhancedRunner
    .sessionService()
    .createSession("Enhanced Currency Flow", "trader")
    .blockingGet()

In [20]:
println("Agent response ->")
runBlocking {
    var events = enhancedRunner.runAsync("trader", newSession.id(), newUserMsg)
        .asFlow()
        .toList()

    events.forEach {
        println(it.stringifyContent())
    }
}

Agent response ->
Function Call: FunctionCall{id=Optional[adk-d9f739bc-5412-4fe1-9b8b-f216a12b8ba6], args=Optional[{arg0=Bank Transfer}], name=Optional[getFeeForPaymentMethod]}
Function Response: FunctionResponse{willContinue=Optional.empty, scheduling=Optional.empty, parts=Optional.empty, id=Optional[adk-d9f739bc-5412-4fe1-9b8b-f216a12b8ba6], name=Optional[getFeeForPaymentMethod], response=Optional[{status=success, fee_percentage=0.01}]}
Function Call: FunctionCall{id=Optional[adk-a652bef9-23f3-41d0-8b61-0b598bbc2918], args=Optional[{arg0=USD, arg1=INR}], name=Optional[getExchangeRate]}
Function Response: FunctionResponse{willContinue=Optional.empty, scheduling=Optional.empty, parts=Optional.empty, id=Optional[adk-a652bef9-23f3-41d0-8b61-0b598bbc2918], name=Optional[getExchangeRate], response=Optional[{status=success, rate=83.58}]}
Function Call: FunctionCall{id=Optional[adk-829d4a51-ea79-4979-ae4f-e9df1a0d0a4d], args=Optional[{request=fee_percentage = 0.01
original_amount = 1250
exch

In [21]:
fun showPythonCodeAndResult(responseEvents: List<Event>) {
    for (event in responseEvents) {
        val responseCodeMap = event.content().orElse(null)
            ?.parts()?.orElse(null)
            ?.firstOrNull()
            ?.functionResponse()?.orElse(null)
            ?.response()

        if (responseCodeMap != null) {
            val resultValue = responseCodeMap.get()["result"] as? String
            if (resultValue != null && resultValue != "```") {
                if (resultValue.contains("tool_code")) {
                    println(
                        "Generated Python Code >> ${resultValue.replace("tool_code", "")}"
                    )
                } else {
                    println("Generated Python Response >> $resultValue")
                }
            }
        }
    }
}

println("✅ Helper function defined.")

✅ Helper function defined.


In [22]:
runBlocking {
    var events = enhancedRunner.runAsync("trader", newSession.id(), newUserMsg)
        .asFlow()
        .toList()
    showPythonCodeAndResult(events)
}

Generated Python Response >> ```python
fee_percentage = 0.01
original_amount = 1250
exchange_rate = 83.58
fee_amount = original_amount * fee_percentage
amount_after_fee = original_amount - fee_amount
converted_amount = amount_after_fee * exchange_rate
print(f"Fee Percentage: {fee_percentage*100:.2f}%")
print(f"Fee Amount: {fee_amount:.2f} USD")
print(f"Amount after fee: {amount_after_fee:.2f} USD")
print(f"Exchange Rate: {exchange_rate}")
print(f"Final Converted Amount: {converted_amount:.2f} INR")
```


In [23]:
// Well it looks like unlike the Python counterpart, Java/Kotlin doesn't seem able to even call the code generated by the code executor?